# separte workflow for coregistration of anatomical image to mni template

In [1]:
# import modules
from os.path import join as opj
import os
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.ants import Registration
from nipype.interfaces.fsl import Info, BET

/home/mirjam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
experiment_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/' # location of experiment folder
subject_list = ['sub-00'] # create the subject_list variable
# run_list = ['1', '2', '3', '4', '5', '6', '7', '8']

output_dir = 'output_reg' # name of output folder
working_dir = 'working_reg' # name of working directory

number_of_slices = 25   # number of slices in volume
TR = 1.5    # time of repetition of volume
fwhm_size = 2 # smoothing kernel

# location of template file for coregistration to mni template (full path needed, error otherwise)
template = Info.standard_image('/home/mirjam/Desktop/Masterarbeit/Auswertung/mni_icbm152_nlin_asym_09c/1mm_T1.nii.gz')

In [3]:
# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'anat': 'derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brain.mgz'} # anatomical images
        
selectfiles = Node(SelectFiles(templates,
                                base_directory=experiment_dir),
                   name="selectfiles")

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]


# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

In [4]:
# BET - Skullstrip mask Image
bet_temp = Node(BET(frac=0.5,
                    robust=True,
                    in_file=template,
                    output_type='NIFTI_GZ'),
                name="bet_temp")

# antsRegistration - coregister t1 on mni template
registrationtemp = Node(Registration(args='--float',
                            collapse_output_transforms=True,
                            initial_moving_transform_com=True,   # no additional connection in workflow needed
                            num_threads=4,
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            sigma_units=['vox']*3,
                            transforms=['Rigid', 'Affine', 'SyN'],
                            terminal_output='file',
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            convergence_threshold=[1e-06],
                            convergence_window_size=[10],
                            metric=['MI', 'MI', 'CC'],
                            metric_weight=[1.0]*3,
                            number_of_iterations=[[1000, 500, 250, 100],
                                                  [1000, 500, 250, 100],
                                                  [100, 70, 50, 20]],
                            radius_or_number_of_bins=[32, 32, 4],
                            sampling_percentage=[0.25, 0.25, 1],
                            sampling_strategy=['Regular',
                                               'Regular',
                                               'None'],
                            shrink_factors=[[8, 4, 2, 1]]*3,
                            smoothing_sigmas=[[3, 2, 1, 0]]*3,
                            transform_parameters=[(0.1,),
                                                  (0.1,),
                                                  (0.1, 3.0, 0.0)],
                            use_histogram_matching=True,
                            write_composite_transform=True),
               name='registrationtemp')

In [8]:
# Create a preprocessing workflow
reg_nc2u = Workflow(name='reg_nc2u')
reg_nc2u.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow - aparc / ribbon mask 
reg_nc2u.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (selectfiles, registrationtemp, [('anat', 'moving_image')]),
                  (bet_temp, registrationtemp, [('out_file', 'fixed_image')]),
                  (registrationtemp, datasink, [('warped_image', 'registrationtemp.@warped_image'),
                                                ('inverse_warped_image', 'registrationtemp.@inverse_warped_image'),
                                                ('inverse_composite_transform', 'registrationtemp.@inverse_composite_transform'),
                                                ('composite_transform', 'registrationtemp.@composite_transform')]),
                      ])

In [24]:
#### run the workflow using multiple cores ####
reg_nc2u.run('MultiProc', plugin_args={'n_procs':4})

180717-16:55:28,871 workflow INFO:
	 Workflow reg_nc2u settings: ['check', 'execution', 'logging', 'monitoring']
180717-16:55:28,883 workflow INFO:
	 Running in parallel.
180717-16:55:28,886 workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 10.45/10.45, Free processors: 4/4.
180717-16:55:28,949 workflow INFO:
	 [Node] Outdated cache found for "reg_nc2u.selectfiles".
180717-16:55:28,951 workflow INFO:
	 [Job 1] Cached (reg_nc2u.bet_mask).
180717-16:55:28,951 workflow INFO:
	 [Node] Setting-up "reg_nc2u.selectfiles" in "/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/working_reg/reg_nc2u/_subject_id_sub-00/selectfiles".
180717-16:55:28,959 workflow INFO:
	 [Node] Outdated cache found for "reg_nc2u.selectfiles".
180717-16:55:28,974 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
180717-16:55:28,988 workflow INFO:
	 [Node] Finished "reg_nc2u.selectfiles".
180717-16:55:30,898 workflow INFO:
	 [Job 0] Completed (reg_nc2u.selectfi

Process NonDaemonPoolWorker-27:
Process NonDaemonPoolWorker-30:
Process NonDaemonPoolWorker-29:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/mirjam/anaconda3/lib/python3.6/multiprocessing/process.py", line 25

KeyboardInterrupt: 